In [1]:
import psycopg2
import time
import pandas as pd

In [2]:
# Параметры подключеник к БД
conn_params = {
    "user": "your_username",
    "password": "your_password",
    "host": "postgres",
    "port": "5432",
    "database": "docker_postgres_db",
}

In [3]:
def export_to_csv(table_name, file_path, conn_params):
    """
    Транзакция: экспорт данных в CSV из таблицы БД и логирование процесса в LOGS.LOGS,
    откат транзакции в случае ошибки.

    Parameters:
    - table_name (str): Таблица в БД для экспорта данных.
    - file_path (str): Путь к CSV (абсолютный).
    - conn_params (dict): Параметры подключеник к БД.

    Returns:
    None
    """
    conn = None
    query = f"COPY {table_name} TO '{file_path}' WITH CSV HEADER;"
    start_time = time.strftime('%Y-%m-%d %H:%M:%S')
    try:
        conn = psycopg2.connect(**conn_params)
        cursor = conn.cursor()
        
        cursor.execute("BEGIN TRANSACTION ISOLATION LEVEL READ COMMITTED;")

        cursor.execute(query)
        rows_count = cursor.rowcount
        conn.commit()
        
        end_time = time.strftime('%Y-%m-%d %H:%M:%S')
        status = 'Export to csv finished'
        error_message = None

    except (Exception, psycopg2.DatabaseError) as error:
        print(f"Error: {error}")
        if conn:
            conn.rollback()

        end_time = time.strftime('%Y-%m-%d %H:%M:%S')
        status = 'Export to csv failed'
        rows_count = None
        error_message = str(error)

    finally:
        if conn:
            cursor.execute(
                "INSERT INTO LOGS.LOGS (table_name, start_time, end_time, status, rows_count, file_path, error_message) "
                "VALUES (%s, %s, %s, %s, %s, %s, %s)",
                (table_name, start_time, end_time, status, rows_count, file_path, error_message)
            )
            conn.commit()
            cursor.close()
            conn.close()

In [4]:
def import_from_csv(table_name, file_path, conn_params, truncate=False):
    """
    Транзакция: импорт данных из CSV в таблицу БД и логирование процесса в LOGS.LOGS,
    откат транзакции в случае ошибки.

    Parameters:
    - table_name (str): Таблица в БД для импорта данных.
    - file_path (str): Путь к CSV(абсолютный или относительный).
    - conn_params (dict): Параметры подключеник к БД.
    - truncate (bool): Если True, очистить таблицу перед копированием данных.

    Returns:
    None
    """
    conn = None
    query = f"COPY {table_name} FROM '{file_path}' DELIMITER ',' CSV HEADER;"
    start_time = time.strftime('%Y-%m-%d %H:%M:%S')
    try:
        conn = psycopg2.connect(**conn_params)
        cursor = conn.cursor()
        
        cursor.execute("BEGIN TRANSACTION ISOLATION LEVEL READ COMMITTED;")

        if truncate == True:
            cursor.execute(f"TRUNCATE {table_name};")

        cursor.execute(query)
        rows_count = cursor.rowcount
        conn.commit()
        
        end_time = time.strftime('%Y-%m-%d %H:%M:%S')
        status = 'Import from csv finished'
        error_message = None

    except (Exception, psycopg2.DatabaseError) as error:
        print(f"Error: {error}")
        if conn:
            conn.rollback()

        end_time = time.strftime('%Y-%m-%d %H:%M:%S')
        status = 'Import from csv failed'
        rows_count = None
        error_message = str(error)

    finally:
        if conn:
            cursor.execute(
                "INSERT INTO LOGS.LOGS (table_name, start_time, end_time, status, rows_count, file_path, error_message) "
                "VALUES (%s, %s, %s, %s, %s, %s, %s)",
                (table_name, start_time, end_time, status, rows_count, file_path, error_message)
            )
            conn.commit()
            cursor.close()
            conn.close()

In [5]:
export_to_csv('dm.dm_f101_round_f', '/var/lib/postgresql/data/pgdata/data/f101_round_f_export.csv', conn_params)
export_to_csv('dm.dm_f101_round_f', 'data/f101_round_f_export.csv', conn_params) # проверка exception

Error: relative path not allowed for COPY to file



In [6]:
import_from_csv('dm.dm_f101_round_f_v2', 'data/f101_round_f_export.csv', conn_params, truncate=True)
import_from_csv('dm.dm_f101_round_f_v2', 'data/f101_round_f_export2.csv', conn_params, truncate=True) # проверка exception

Error: could not open file "data/f101_round_f_export2.csv" for reading: No such file or directory
HINT:  COPY FROM instructs the PostgreSQL server process to read a file. You may want a client-side facility such as psql's \copy.

